[주제] 테크 뉴스레터 작성하기

# CrewAI와 Tool 결합

In [4]:
import warnings
warnings.filterwarnings('ignore')

- Serper : Google Search API 키 발급
    - https://serper.dev/

In [5]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# 라이브러리 임포트

### crewai_tools
- SerperDevTool : Serper api를 활용한 결과물 받는 도구, 구글 웹검색 도구
- WebsiteSearchTool : 특정 키워드와 특정 웹사이트를 주면, 해당 웹 사이트에서 키워드와 관련이 있는 부부만 가져옴.
- ScrapeWebsiteTool : 어떤 URL을 주면 해당 url에 있는 텍스트를 스크래핑해서 가져오는 도구

In [6]:
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# 필요한 객체 생성

In [7]:
# llm 객체
llm = ChatOpenAI(model="gpt-4o-mini")

# Tool
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

# Agent 정의

In [15]:
# 테크 트렌드 연구원
researcher = Agent(
    name="tech_trend_researcher",
    role = "테크 트렌드 연구원",
    goal="인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2026년 1월입니다.",
    backstory="기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.",
    tools=[search_tool, web_rag_tool],
    # verbose=True,
    max_iter=5,  # 해당 Agent가 주어진 task를 수행할 때 반복 실행할 횟수를 제한하는 것임(웹검색을 5번 실행, 비용과 연결되어 있음.)
    llm=llm
)

- allow_delegation=False
    - 해당 agnet가 task를 수행할 때 혼자 힘으로 하기 힘들 때 다른 agnet에 위임하는 설정
    - detault=True
    - False로 하지 않으면 수행중에 researcher에게 작업을 넘기는 경우가 발생하는데 이것을 발생하지 않도록 제한하기 위함

In [ ]:
# 뉴스레터 작성 agnet
writer = Agent(
    name="tech_newsletter_writer",
    role="뉴스레터 작성자",
    goal="최신 AI 기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2026년 1월입니다.",
    backstory="기술에 대한 열정을 가진 숙련된 작가입니다.",
    # verbose=True,
    allow_delegation=False,  
    llm=llm
)

# Task 정의

In [17]:
research = Task(
    name="tech_trend_research",
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output="AI 업계에서 가장 주목받는 3대 기술 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

In [ ]:
write = Task(
    name="tech_newsletter_writing",
    description="""
    테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
    테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.
    """,
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터", # 결과물 형식
    agent=writer,
    output_file=r"./outputs/new_post.md" # the final blog post will be saved here
)

# Crew 정의

In [21]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    # verbose=False,
    verbose=True,
    process=Process.sequential,
    output_log_file="logs/crew_run.json",   # 👈 JSON 형태로 모든 스텝이 저장됩니다.
)

In [22]:
# 폴더가 없을 경우 생성하기
import os
os.makedirs("./outputs", exist_ok=True)
os.makedirs("./logs", exist_ok=True)


# Crew 실행

In [23]:
import json

result = crew.kickoff()

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  857c1ea1-dd0c-42d0-bc42-0d3d929edc55                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: tech_trend_research                                                                                      │
│  ID: b31cd628-4879-440c-9afa-4210ad6c8b2d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 테크 트렌드 연구원                                                                                      │
│                                                                                                                 │
│  Task: AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: Search the internet with Serper                                                                          │
│  Args: {"search_query": "2026 AI industry trends important developments"}                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': '2026 AI industry trends important developments', 'type': 'search', 'num': 10,      │
│  'engine': 'google'}, 'organic': [{'title': 'The trends that will shape AI and tech in 2026', 'link':           │
│  'https://www.ibm.com/think/news/ai-tech-trends-predictions-2026', 'snippet': 'Reporter Anabelle Nicoud spoke   │
│  to several experts across AI, security, quantum and beyond to better understand where tech will take us in     │
│  2026.', 'position': 1}, {'title': "What's next in AI: 7 trends to watch in 2026", 'link':                      │
│  'https://news.microsoft.com/source/features/ai/whats-next-in-ai-7-trends-to-watch-in-2026/', 'snippet':        │
│  'Seven AI trends to watch in 2026 will make AI a true partner — boosting teamwork, security, research          │
│  momentum and infrastructure efficiency.', 'position': 2}, {'title': 'Five Trends in AI and Data Science for    │
│  2026', 'link': 'https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2026/', 'snippet':  │
│  "From the AI bubble to GenAI's rise as an organizational tool, these are the 2026 AI trends to watch. Explore  │
│  new data and advice from AI experts ...", 'position': 3}, {'title': "What's next for AI in 2026 | MIT          │
│  Technology ...", 'link': 'https://www.technologyreview.com/2026/01/05/1130662/whats-next-for-ai-in-2026/',     │
│  'snippet': 'In 2026, expect more Silicon Valley apps to quietly ship on top of Chinese open models, and look   │
│  for the lag between Chinese releases and the ...', 'position': 4}, {'title': 'Tech Trends 2026 | Deloitte      │
│  Insights', 'link': 'https://www.deloitte.com/us/en/insights/topics/technology-management/tech-trends.html',    │
│  'snippet': 'AI is restructuring tech organizations, making them leaner, faster, and more strategic. Only 1%    │
│  of IT leaders surveyed by Deloitte reported that ...', 'position': 5}, {'title': 'Five Significant Tech        │
│  Trends That Will Feature in 2026', 'link':                                                                     │
│  'https://internationalbanker.com/technology/five-significant-tech-trends-that-will-feature-in-2026/',          │
│  'snippet': 'In contrast, 2026 may well be the year of agentic...                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 뉴스레터 작성자                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│  테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.                       │
│  테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 뉴스레터 작성자                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # 최신 AI 기술 트렌드 뉴스레터 (2026년 1월)                                                                    │
│                                                                                                                 │
│  안녕하세요, AI 기술 애호가 여러분! 🤖 2026년의 첫 달이 밝았습니다. AI 업계는 빠르게 변화하고 있으며, 올해      │
│  주목할 만한 트렌드들을 소개해 드리려고 합니다. 기발한 아이디어와 혁신으로 가득한 세상에서, 여러분의 기업       │
│  전략에도 큰 도움이 될 만한 정보들입니다!                                                                       │
│                                                                                                                 │
│  ## 에이전틱 AI의 혁명 🌟                                                                                       │
│  현재 AI의 새로운 아이콘, 에이전틱 AI가 대세로 떠오르고 있습니다! 이 시스템들은 단순히 질문에 대한 답변을 넘어  │
│  사용자의 니즈를 실시간으로 파악하고 복잡한 문제들을 손쉽게 해결해주는 매력적인 존재입니다. 예를 들어, AI가     │
│  기업의 워크플로우를 효율적으로 관리하고 결함을 조기에 발견할 수 있도록 돕는 모습을 상상해 보세요. 이제는 AI가  │
│  책임감을 가지고 결정에 영향을 미치는 방향으로 나아가고 있으며, 이는 비즈니스 운영 상의 ROI를 새롭게            │
│  재구성하는 기회를 제공합니다. 에이전틱 AI는 오늘날 기업 전략에서 빼놓을 수 없는 핵심 요소로 부각되고           │
│  있습니다!                                                                                                      │
│                                                                                                                 │
│  ## 오픈 소스 AI의 협업 시대 🌐                                                                                 │
│  2026년에는 오픈 소스 AI가 더욱 크고 강력해질 것이라는 예측이 나옵니다. 특히, 다양한 국가들이 각기 다른 모델    │
│  개발에 박차를 가하고 있으며, 중국의 다국어 및 추론 조정 모델들은 그 선두에서 활약하고 있습니다. 데이터 주권을  │
│  고려해야 하는 시대, 오픈 소스 AI는 통합성과 투명성을 기반으로 한 강력한 협력 환경을 만들어낼 것입니다. 개방된  │
│  데이터 파이프라인과 보안 감사 모델이 결합되어 AI의 성능을 더욱 끌어올릴 것으로 기대됩니다.                     │
│                                                                                                                 │
│  ## 신뢰성과 보안의 시대 ☁️🔒                                                                                    │
│  AI의 채택이 증가함에 따라 신뢰성과 보안의 중요성 역시 늘어나고 있습니다. 데이터 유출이나 접근 손실과 같은      │
│  리스크가 확대되는 가운데, AI 주권은 필수적으로 고려해야 할 요소입니다. 특히, 모든 AI 시스템의 데이터 관리가    │
│  외부 의존 없이 이루어져야 하며, 이를 통해 기업들은 더욱 강화된 보안을 구축할 수 있습니다. AI 제품은 그 데이터  │
│  처리 과정을 투명하게 보여줘야 하며, 이는 궁극적으로 AI의 신뢰성을 높이는 방향으로 나아가고 있습니다.           │
│                                                                                                                 │
│  2026년, 혁신적인 AI 트렌드는 여러분의 비즈니스와 전략적 기회를 더욱 확장시킬 것입니다. 이 흥미진진한 변화의    │
│  물결에 함께 동참해 보시길 바랍니다! 추가로 궁금한 점이나 논의하고 싶은 주제가 있다면, 언제든지 저희에게        │
│  문의해 주세요. 다음 뉴스레터에서 더 많은 인사이트로 찾아뵙겠습니다! 🌈                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  tech_newsletter_writing                                                                                        │
│  Agent:                                                                                                         │
│  뉴스레터 작성자                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  857c1ea1-dd0c-42d0-bc42-0d3d929edc55                                                                           │
│  Final Output: # 최신 AI 기술 트렌드 뉴스레터 (2026년 1월)                                                      │
│                                                                                                                 │
│  안녕하세요, AI 기술 애호가 여러분! 🤖 2026년의 첫 달이 밝았습니다. AI 업계는 빠르게 변화하고 있으며, 올해      │
│  주목할 만한 트렌드들을 소개해 드리려고 합니다. 기발한 아이디어와 혁신으로 가득한 세상에서, 여러분의 기업       │
│  전략에도 큰 도움이 될 만한 정보들입니다!                                                                       │
│                                                                                                                 │
│  ## 에이전틱 AI의 혁명 🌟                                                                                       │
│  현재 AI의 새로운 아이콘, 에이전틱 AI가 대세로 떠오르고 있습니다! 이 시스템들은 단순히 질문에 대한 답변을 넘어  │
│  사용자의 니즈를 실시간으로 파악하고 복잡한 문제들을 손쉽게 해결해주는 매력적인 존재입니다. 예를 들어, AI가     │
│  기업의 워크플로우를 효율적으로 관리하고 결함을 조기에 발견할 수 있도록 돕는 모습을 상상해 보세요. 이제는 AI가  │
│  책임감을 가지고 결정에 영향을 미치는 방향으로 나아가고 있으며, 이는 비즈니스 운영 상의 ROI를 새롭게            │
│  재구성하는 기회를 제공합니다. 에이전틱 AI는 오늘날 기업 전략에서 빼놓을 수 없는 핵심 요소로 부각되고           │
│  있습니다!                                                                                                      │
│                                                                                                                 │
│  ## 오픈 소스 AI의 협업 시대 🌐                                                                                 │
│  2026년에는 오픈 소스 AI가 더욱 크고 강력해질 것이라는 예측이 나옵니다. 특히, 다양한 국가들이 각기 다른 모델    │
│  개발에 박차를 가하고 있으며, 중국의 다국어 및 추론 조정 모델들은 그 선두에서 활약하고 있습니다. 데이터 주권을  │
│  고려해야 하는 시대, 오픈 소스 AI는 통합성과 투명성을 기반으로 한 강력한 협력 환경을 만들어낼 것입니다. 개방된  │
│  데이터 파이프라인과 보안 감사 모델이 결합되어 AI의 성능을 더욱 끌어올릴 것으로 기대됩니다.                     │
│                                                                                                                 │
│  ## 신뢰성과 보안의 시대 ☁️🔒                                                                                    │
│  AI의 채택이 증가함에 따라 신뢰성과 보안의 중요성 역시 늘어나고 있습니다. 데이터 유출이나 접근 손실과 같은      │
│  리스크가 확대되는 가운데, AI 주권은 필수적으로 고려해야 할 요소입니다. 특히, 모든 AI 시스템의 데이터 관리가    │
│  외부 의존 없이 이루어져야 하며, 이를 통해 기업들은 더욱 강화된 보안을 구축할 수 있습니다. AI 제품은 그 데이터  │
│  처리 과정을 투명하게 보여줘야 하며, 이는 궁극적으로 AI의 신뢰성을 높이는 방향으로 나아가고 있습니다.           │
│                                                                                                                 │
│  2026년, 혁신적인 AI 트렌드는 여러분의 비즈니스와 전략적 기회를 더욱 확장시킬 것입니다. 이 흥미진진한 변화의    │
│  물결에 함께 동참해 보시길 바랍니다! 추가로 궁금한 점이나 논의하고 싶은 주제가 있다면, 언제든지 저희에게        │
│  문의해 주세요. 다음 뉴스레터에서 더 많은 인사이트로 찾아뵙겠습니다! 🌈                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# 마크다운으로 표시

In [45]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# 최신 AI 기술 트렌드 뉴스레터 (2026년 1월)

안녕하세요, 테크 애호가 여러분! 2026년이 밝았습니다. 새로운 해, 새로운 시작! 오늘은 인공지능(AI) 분야에서 가장 주목할 만한 기술 트렌드를 살펴보겠습니다. AI가 세상을 어떻게 변화시키고 있는지, 그리고 이것이 우리 삶에 어떤 영향을 미치는지를 아는 것은 불가결한 요소가 되었죠. 그럼, 주요 트렌드를 한 번 알아볼까요?

## Generative AI의 혁신

최근 몇 년간 우리는 Generative AI의 눈부신 발전을 목격하였습니다. 이 기술은 이제 단순한 호기심의 대상이 아니라, 텍스트, 이미지, 음악 등 다양한 콘텐츠를 창작하는 데에 필수적인 도구로 자리 잡았습니다. OpenAI의 GPT 모델과 DALL-E와 Midjourney와 같은 이미지 생성 모델들은 창작자들에게 무한한 아이디어의 원천을 제공하고 있답니다. 이러한 변화는 콘텐츠 제작의 패러다임을 바꿔놓고 있으며, 개인 맞춤형 서비스를 통해 조금 더 진화된 비즈니스 모델을 탐험할 수 있는 기회를 제공하고 있습니다.

## 윤리적 규범과 규제의 필요성

AI의 급속한 발전에 따라 윤리적 사회적 문제도 불거지고 있습니다. 개인 정보 보호, 저작권, 알고리즘의 편향성 등 여러 이슈들이 대두되고 있죠. 올해는 정부와 기업이 협력하여 이 문제를 적극적으로 해결하려는 움직임이 강화될 것으로 보입니다. AI 개발자들에게는 윤리적 기준을 준수하는 것만큼 중요한 과제가 없겠죠. 이는 기업의 신뢰도를 높여 소비자들로부터 긍정적인 반응을 불러올 준비가 되어 있다는 신호입니다!

## AI와 엣지 컴퓨팅의 융합

사물인터넷(IoT) 기기의 확산과 함께, 엣지 컴퓨팅 기술이 더욱 중요해지고 있습니다. 이제 AI는 데이터 전송의 지연을 최소화하면서 엣지 디바이스에서 직접 처리할 수 있게 되었답니다. 이러한 혁신은 산업 자동화, 자율주행차 및 스마트 홈 기술 등에서 운영 효율성을 극대화하고, 비용 절감의 길을 열어줍니다. 뿐만 아니라, 엣지 AI는 데이터 보안을 강화하고 네트워크 대역폭의 효율성을 높이며, 데이터 처리 시간 역시 대폭 단축시키는데 기여할 것으로 보입니다.

이렇게 2026년, AI는 우리 생활의 모든 면에서 변화를 주도하고 있습니다. 이러한 기술 흐름을 정확히 이해하고 잘 활용하는 것이 회사의 생존 전략이 될 것입니다. AI가 가져올 미래의 변화를 기대하시고, 매력적인 기술의 세계에 함께 발을 담가보시길 바랍니다!

[tip] log 후처리
- logs/crew_run.json의 로그 결과에서 한글화 하기 위한 JSON 변환하기

In [24]:
import json

with open("logs/crew_run.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("logs/crew_run_readable.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("crew_run_readable.json 생성 완료")


crew_run_readable.json 생성 완료
